# Data Merge for Wikipedia Politicians Analysis

## Import Libraries and Load Data

In [8]:
import pandas as pd
import numpy as np

population_df = pd.read_csv("intermediate_data/population_by_country_with_region.csv") 
politicians_df = pd.read_csv("intermediate_data/politicians_with_quality_and_revisions.csv") # change this and delete article scores

## Data Cleaning and Preprocessing

This cell performs initial data cleaning:
1. Checks for missing values in both datasets.
2. Removes rows with missing values.
3. Checks for and removes duplicate entries in both datasets.

In [9]:
# Check for missing values
print("Missing values in politicians_df dataset:")
print(politicians_df.isnull().sum())
print("\nMissing values in population_df dataset:")
print(population_df.isnull().sum())

# Remove rows with missing values
politicians_df = politicians_df.dropna()
population_df = population_df.dropna()

# Check for duplicates
print("\nDuplicates in poly dataset:")
print(politicians_df.duplicated().sum())
print("\nDuplicates in population_df dataset:")
print(population_df.duplicated().sum())

# Remove duplicates
politicians_df = politicians_df.drop_duplicates()
population_df = population_df.drop_duplicates()

Missing values in politicians_df dataset:
name            0
url             0
country         0
quality        12
revision_id     8
dtype: int64

Missing values in population_df dataset:
country       0
Population    0
region        0
dtype: int64

Duplicates in poly dataset:
0

Duplicates in population_df dataset:
0


## Handling outlier like korea and guineabissau


This section handles specific outliers and standardizes country names:
1. Defines a function to clean and standardize country names.
2. Applies the cleaning function to both datasets.
3. Combines the population data for North and South Korea into a single entry.

In [10]:
# Function to clean country names
def clean_country_name(name):
    name = name.strip().lower()
    # Handle specific cases
    name_map = {
        "korea (north)": "korea",
        "korea (south)": "korea",
        "korea, south": "korea",
        "korean": "korea",
        "guineabissau": "guinea-bissau"
    }
    return name_map.get(name, name)

# Clean country names in both datasets
politicians_df['country'] = politicians_df['country'].apply(clean_country_name)
population_df['country'] = population_df['country'].apply(clean_country_name)

# Combine North and South Korea population
korea_data = population_df[population_df['country'] == 'korea']
korea_total_pop = korea_data['Population'].sum()

# Remove individual Korea entries and add combined Korea
population_df = population_df[population_df['country'] != 'korea']
new_korea = pd.DataFrame({'country': ['korea'], 'Population': [korea_total_pop]})
population_df = pd.concat([population_df, new_korea], ignore_index=True)

## Merging the Datasets

This cell merges the two datasets:
1. Performs an inner join on the 'country' field.
2. Identifies and saves countries that didn't match between the datasets.
3. Keeps only the successfully matched data.
4. Selects and renames relevant columns for the final dataset.

In [11]:
# Merge the two datasets on the 'country' field
merged_df = pd.merge(politicians_df, population_df, on='country', how='inner', indicator=True)

# Save the countries that did not match between the two datasets
no_match_df = merged_df[merged_df['_merge'] != 'both'][['country']].drop_duplicates()
no_match_df.to_csv('output_data/wp_countries-no_match.txt', index=False, header=False)

# Keep only the successfully matched data
matched_df = merged_df[merged_df['_merge'] == 'both']

# Select relevant columns for the final DataFrame
final_df = matched_df[['country', 'region', 'Population', 'name', 'revision_id', 'quality']]

# Rename columns as per the assignment's requirement
final_df = final_df.rename(columns={
    'name': 'article_title',
    'quality': 'article_quality',
    'Population': 'population'
})

## Save the Final Dataset and Summary

In [12]:
# Save the final merged dataset to a CSV file
final_df.to_csv('output_data/wp_politicians_by_country.csv', index=False)

# Summary
print("\nSummary of Merged Data:")
print(f"Final merged dataset: {final_df.shape[0]} rows")
print(f"Unmatched countries saved to 'wp_countries-no_match.txt'.")
print(f"Final merged dataset saved to 'wp_politicians_by_country.csv'.")


Summary of Merged Data:
Final merged dataset: 7143 rows
Unmatched countries saved to 'wp_countries-no_match.txt'.
Final merged dataset saved to 'wp_politicians_by_country.csv'.
